# Rag Avancé

## Imports

In [ ]:
from typing import TypedDict, List, Literal
from langgraph.graph import StateGraph, END
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.retrievers import BM25Retriever
from rank_bm25 import BM25Okapi
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


class AdvancedRAGState(TypedDict):
    """État avec fonctionnalités avancées"""
    original_question: str
    rewritten_queries: List[str]
    query_type: str  # "simple", "multi_doc", "comparative"
    documents: List[Document]
    retrieved_docs_semantic: List[Document]
    retrieved_docs_lexical: List[Document]
    merged_docs: List[Document]
    reranked_docs: List[Document]
    answer: str
    confidence_score: float
    validation_result: dict
    cache_hit: bool
    error: str | None

## Configuration

In [ ]:
# Embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# LLM principal et de validation
llm_main = ChatOpenAI(model="gpt-4o", temperature=0)
llm_validator = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Cache simple (en production: Redis)
query_cache = {}

# Nodes

In [ ]:
def analyze_query(state: AdvancedRAGState) -> AdvancedRAGState:
    """Agent d'analyse de la requête"""
    print("Analyse de la requête...")
    
    query = state["original_question"]
    
    # Prompt pour classifier la requête
    classifier_prompt = ChatPromptTemplate.from_messages([
        ("system", "Classifie la requête en: 'simple' (une info), 'multi_doc' (plusieurs sources), ou 'comparative' (comparaison)"),
        ("user", "Question: {question}\n\nRéponds uniquement par: simple, multi_doc ou comparative")
    ])
    
    chain = classifier_prompt | llm_main
    result = chain.invoke({"question": query})
    query_type = result.content.strip().lower()
    
    state["query_type"] = query_type if query_type in ["simple", "multi_doc", "comparative"] else "simple"
    print(f"Type détecté: {state['query_type']}")
    return state


def check_cache(state: AdvancedRAGState) -> AdvancedRAGState:
    """Vérifie si la réponse est en cache"""
    print("Vérification du cache...")
    
    query_key = state["original_question"].lower().strip()
    
    if query_key in query_cache:
        print("Cache HIT!")
        cached_result = query_cache[query_key]
        state["answer"] = cached_result["answer"]
        state["confidence_score"] = cached_result["confidence"]
        state["cache_hit"] = True
    else:
        print("Cache MISS")
        state["cache_hit"] = False
    
    return state


def should_use_cache(state: AdvancedRAGState) -> Literal["cached", "process"]:
    """Décision: utiliser le cache ou continuer le traitement"""
    return "cached" if state.get("cache_hit") else "process"


def rewrite_query(state: AdvancedRAGState) -> AdvancedRAGState:
    """Query rewriting: génère plusieurs variantes de la question"""
    print("Réécriture de la requête...")
    
    query = state["original_question"]
    
    rewriter_prompt = ChatPromptTemplate.from_messages([
        ("system", """Tu es un expert en reformulation de questions pour améliorer la recherche.
Génère 2-3 variantes de la question qui utilisent:
- Des synonymes
- Des formulations différentes
- Des termes techniques si pertinent

Format: une variante par ligne, sans numérotation."""),
        ("user", "Question originale: {question}")
    ])
    
    chain = rewriter_prompt | llm_main
    result = chain.invoke({"question": query})
    
    variants = [line.strip() for line in result.content.split("\n") if line.strip()]
    state["rewritten_queries"] = [query] + variants[:2]
    
    print(f"{len(state['rewritten_queries'])} variantes créées:")
    for i, q in enumerate(state["rewritten_queries"], 1):
        print(f"   {i}. {q}")
    
    return state


def semantic_search(state: AdvancedRAGState) -> AdvancedRAGState:
    """Recherche sémantique avec embeddings"""
    print("Recherche sémantique...")
    
    vectorstore = state.get("vectorstore")
    if not vectorstore:
        state["retrieved_docs_semantic"] = []
        return state
    
    all_docs = []
    seen_content = set()
    
    # Recherche pour chaque variante de la question
    for query in state["rewritten_queries"]:
        docs = vectorstore.similarity_search_with_score(query, k=3)
        for doc, score in docs:
            content_hash = hash(doc.page_content)
            if content_hash not in seen_content:
                doc.metadata["similarity_score"] = score
                all_docs.append(doc)
                seen_content.add(content_hash)
    
    state["retrieved_docs_semantic"] = all_docs[:5]
    print(f"{len(state['retrieved_docs_semantic'])} documents sémantiques")
    return state


def lexical_search(state: AdvancedRAGState) -> AdvancedRAGState:
    """Recherche lexicale BM25"""
    print("Recherche lexicale (BM25)...")
    
    documents = state.get("documents", [])
    if not documents:
        state["retrieved_docs_lexical"] = []
        return state
    
    # Création du retriever BM25
    bm25_retriever = BM25Retriever.from_documents(documents)
    bm25_retriever.k = 3
    
    # Recherche pour la question originale
    docs = bm25_retriever.get_relevant_documents(state["original_question"])
    
    for doc in docs:
        doc.metadata["search_type"] = "lexical"
    
    state["retrieved_docs_lexical"] = docs
    print(f"{len(docs)} documents lexicaux")
    return state


def hybrid_merge(state: AdvancedRAGState) -> AdvancedRAGState:
    """Fusionne et déduplique les résultats sémantiques et lexicaux"""
    print("Fusion hybride des résultats...")
    
    semantic_docs = state.get("retrieved_docs_semantic", [])
    lexical_docs = state.get("retrieved_docs_lexical", [])
    
    # Fusion avec déduplication
    merged = {}
    
    # Pondération: sémantique = 0.7, lexical = 0.3
    for doc in semantic_docs:
        content_hash = hash(doc.page_content)
        merged[content_hash] = {
            "doc": doc,
            "score": 0.7 * (1 - doc.metadata.get("similarity_score", 0))
        }
    
    for doc in lexical_docs:
        content_hash = hash(doc.page_content)
        if content_hash in merged:
            merged[content_hash]["score"] += 0.3
        else:
            merged[content_hash] = {"doc": doc, "score": 0.3}
    
    # Tri par score décroissant
    sorted_docs = sorted(
        merged.values(),
        key=lambda x: x["score"],
        reverse=True
    )
    
    state["merged_docs"] = [item["doc"] for item in sorted_docs[:5]]
    print(f"{len(state['merged_docs'])} documents fusionnés")
    return state


def rerank_documents(state: AdvancedRAGState) -> AdvancedRAGState:
    """Reranking avec un LLM pour affiner la pertinence"""
    print("🎯 Reranking des documents...")
    
    docs = state.get("merged_docs", [])
    if not docs:
        state["reranked_docs"] = []
        return state
    
    # Pour une vraie implémentation, utiliser un cross-encoder
    # Ici, simulation simple avec LLM
    
    rerank_prompt = ChatPromptTemplate.from_messages([
        ("system", """Évalue la pertinence de chaque document pour répondre à la question.
Score de 0 à 10. Format: Document X: score"""),
        ("user", """Question: {question}

Documents:
{documents}""")
    ])
    
    docs_text = "\n\n".join([
        f"Document {i+1}:\n{doc.page_content[:300]}..."
        for i, doc in enumerate(docs)
    ])
    
    chain = rerank_prompt | llm_validator
    result = chain.invoke({
        "question": state["original_question"],
        "documents": docs_text
    })
    
    # Parsing simple des scores (en production: plus robuste)
    scores = [7, 6, 5, 4, 3]  # Fallback par défaut
    
    try:
        lines = result.content.split("\n")
        scores = []
        for line in lines:
            if ":" in line and any(str(i) in line for i in range(11)):
                score_part = line.split(":")[-1].strip()
                score = float(''.join(c for c in score_part if c.isdigit() or c == '.'))
                scores.append(score)
    except Exception:
        pass
    
    # Associer scores et trier
    docs_with_scores = list(zip(docs, scores[:len(docs)]))
    docs_with_scores.sort(key=lambda x: x[1], reverse=True)
    
    state["reranked_docs"] = [doc for doc, _ in docs_with_scores[:3]]
    print(f"Top {len(state['reranked_docs'])} documents après reranking")
    return state


def generate_answer_advanced(state: AdvancedRAGState) -> AdvancedRAGState:
    """Génération avancée avec métadonnées"""
    print("Génération de la réponse avancée...")
    
    docs = state.get("reranked_docs", [])
    if not docs:
        state["answer"] = "Je n'ai pas trouvé suffisamment d'informations pertinentes."
        state["confidence_score"] = 0.0
        return state
    
    # Contexte enrichi avec métadonnées
    context_parts = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get("source", "inconnu")
        context_parts.append(f"[Source: {source}]\n{doc.page_content}")
    
    context = "\n\n---\n\n".join(context_parts)
    
    # Prompt avancé avec instructions de citation
    advanced_prompt = ChatPromptTemplate.from_messages([
        ("system", """Tu es un assistant expert qui répond avec précision en citant ses sources.

Instructions:
- Utilise UNIQUEMENT les informations des documents fournis
- Cite la source entre crochets [Source: X] après chaque affirmation
- Si plusieurs sources, compare et synthétise
- Si information manquante, dis-le clairement
- Sois concis mais complet"""),
        ("user", """Documents:\n{context}

Question: {question}

Réponds de manière structurée et cite tes sources.""")
    ])
    
    chain = advanced_prompt | llm_main
    response = chain.invoke({
        "context": context,
        "question": state["original_question"]
    })
    
    state["answer"] = response.content
    state["confidence_score"] = min(len(docs) / 3.0, 1.0)  # Score basé sur nb de docs
    print(f" Réponse générée (confiance: {state['confidence_score']:.2f})")
    return state


def validate_answer(state: AdvancedRAGState) -> AdvancedRAGState:
    """Valide la réponse pour détecter hallucinations"""
    print("Validation de la réponse...")
    
    answer = state.get("answer", "")
    docs = state.get("reranked_docs", [])
    
    if not answer or not docs:
        state["validation_result"] = {"valid": False, "reason": "Données insuffisantes"}
        return state
    
    # Validation avec LLM
    validation_prompt = ChatPromptTemplate.from_messages([
        ("system", """Vérifie si la réponse est fidèle aux documents sources.
Détecte:
- Hallucinations (infos non présentes)
- Contradictions
- Extrapolations excessives

Format JSON: {"valid": true/false, "issues": ["liste"], "confidence": 0-100}"""),
        ("user", """Documents sources:
{context}

Réponse à vérifier:
{answer}

Validation:""")
    ])
    
    context = "\n\n".join([doc.page_content for doc in docs])
    
    chain = validation_prompt | llm_validator
    result = chain.invoke({"context": context, "answer": answer})
    
    # Parsing simple (en production: JSON parser robuste)
    validation = {
        "valid": "true" in result.content.lower() or "valid" in result.content.lower(),
        "confidence": 85,  # Valeur par défaut
        "issues": []
    }
    
    state["validation_result"] = validation
    print(f"Validation: {'✓ Valide' if validation['valid'] else '✗ Problème détecté'}")
    return state


def save_to_cache(state: AdvancedRAGState) -> AdvancedRAGState:
    """Sauvegarde dans le cache si validation OK"""
    print("Sauvegarde en cache...")
    
    if state.get("validation_result", {}).get("valid", False):
        query_key = state["original_question"].lower().strip()
        query_cache[query_key] = {
            "answer": state["answer"],
            "confidence": state["confidence_score"]
        }
        print("Réponse mise en cache")
    else:
        print("Réponse non cachée (validation échouée)")
    
    return state


def route_by_complexity(state: AdvancedRAGState) -> Literal["simple_path", "complex_path"]:
    """Routing intelligent selon la complexité"""
    query_type = state.get("query_type", "simple")
    
    if query_type in ["multi_doc", "comparative"]:
        print("→ Route complexe")
        return "complex_path"
    else:
        print("→ Route simple")
        return "simple_path"

# Graph

In [ ]:
def create_advanced_rag_graph():
    """Crée le graphe RAG avancé avec agents et routing"""
    
    workflow = StateGraph(AdvancedRAGState)
    
    # Phase 1: Analyse et cache
    workflow.add_node("analyze", analyze_query)
    workflow.add_node("cache_check", check_cache)
    workflow.add_node("cached_response", lambda s: s)  # Nœud passthrough
    
    # Phase 2: Query processing
    workflow.add_node("rewrite", rewrite_query)
    
    # Phase 3: Recherche
    workflow.add_node("semantic", semantic_search)
    workflow.add_node("lexical", lexical_search)
    workflow.add_node("merge", hybrid_merge)
    workflow.add_node("rerank", rerank_documents)
    
    # Phase 4: Génération et validation
    workflow.add_node("generate", generate_answer_advanced)
    workflow.add_node("validate", validate_answer)
    workflow.add_node("cache_save", save_to_cache)
    
    # Flux du graphe
    workflow.set_entry_point("analyze")
    workflow.add_edge("analyze", "cache_check")
    
    # Branchement conditionnel: cache ou traitement
    workflow.add_conditional_edges(
        "cache_check",
        should_use_cache,
        {
            "cached": "cached_response",
            "process": "rewrite"
        }
    )
    
    workflow.add_edge("cached_response", END)
    
    # Branchement selon complexité
    workflow.add_conditional_edges(
        "rewrite",
        route_by_complexity,
        {
            "simple_path": "semantic",
            "complex_path": "semantic"  # Les deux passent par semantic d'abord
        }
    )
    
    workflow.add_edge("semantic", "lexical")
    workflow.add_edge("lexical", "merge")
    workflow.add_edge("merge", "rerank")
    workflow.add_edge("rerank", "generate")
    workflow.add_edge("generate", "validate")
    workflow.add_edge("validate", "cache_save")
    workflow.add_edge("cache_save", END)
    
    return workflow.compile()

## Test

In [ ]:
if __name__ == "__main__":
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    
    # Documents d'exemple
    sample_docs = [
        Document(
            page_content="""Notre politique de remboursement:
- Délai de rétractation: 30 jours calendaires
- Remboursement intégral si produit non ouvert et dans emballage d'origine
- Frais de retour à la charge du client sauf produit défectueux
- Remboursement sous 10 jours après réception du retour""",
            metadata={"source": "politique_remboursement.pdf"}
        ),
        Document(
            page_content="""Garanties par forfait:
Pack Basic: 
- 1 an garantie constructeur
- Support par email sous 48h

Pack Premium:
- 3 ans garantie constructeur + 2 ans extension
- Support prioritaire 24/7 par téléphone
- Prêt de matériel en cas de panne

Pack Ultimate:
- 5 ans garantie tous risques (casse, vol, oxydation)
- Remplacement sous 24h garanti
- Support dédié avec interlocuteur unique
- Maintenance préventive annuelle gratuite""",
            metadata={"source": "garanties_forfaits.pdf"}
        ),
        Document(
            page_content="""Délais et modes de livraison:
Standard (gratuit):
- France métropolitaine: 3-5 jours ouvrés
- Corse: 5-7 jours
- DOM-TOM: 10-15 jours

Express (19.90€):
- France: 24-48h
- Non disponible DOM-TOM

Premium (39.90€):
- Livraison le lendemain avant 12h
- France métropolitaine uniquement
- Commande avant 15h""",
            metadata={"source": "livraison_delais.pdf"}
        )
    ]
    
    # Preprocessing des documents
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50
    )
    chunks = splitter.split_documents(sample_docs)
    
    # Création vectorstore
    vectorstore = FAISS.from_documents(chunks, embeddings)
    
    # Création du graphe
    app = create_advanced_rag_graph()
    
    print("\n" + "="*70)
    print("🚀 RAG AVANCÉ - DÉMONSTRATION")
    print("="*70 + "\n")
    
    # Test 1: Question simple
    state1 = {
        "original_question": "Quel est le délai de remboursement ?",
        "rewritten_queries": [],
        "query_type": "",
        "documents": chunks,
        "retrieved_docs_semantic": [],
        "retrieved_docs_lexical": [],
        "merged_docs": [],
        "reranked_docs": [],
        "answer": "",
        "confidence_score": 0.0,
        "validation_result": {},
        "cache_hit": False,
        "error": None,
        "vectorstore": vectorstore
    }
    